## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importando o dataset

In [2]:
data = pd.read_csv('adm_data.csv')
data.rename(columns = {'Chance of Admit ': 'Chance of Admit'}, inplace = True)
data.rename(columns = {'LOR ': 'LOR'}, inplace = True)

FileNotFoundError: [Errno 2] No such file or directory: 'adm_data.csv'

In [ ]:
features = ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research']
label = 'Chance of Admit'

### Mudando a escala dos dados para [0, 1]

Transformaremos [min, max] -> [0, 1] para cada coluna.

In [ ]:
def scale(feature):
    minimum = min(feature)
    maximum = max(feature)
    return (feature - minimum)/(maximum - minimum)

In [ ]:
data_scaled = data.copy()
for feature in features:
    data_scaled[feature] = scale(data[feature])

## Modelo com Regularização para Complexidade

Iremos modificar o modelo implementado anteriormente e adicionar o termo de complexidade à função de custo, definindo-o pela fórmula de regularização $L_{2}$ ($L_{2}$ Regularization formula). Tal termo é dado por:

$L_{2}$ regularization term = $ \sum_{i = 1}^{n} w_{i}^{2}$

Definimos $\lambda$ como o coeficiente de regularização, um hiperparâmetro do modelo que multiplica a função de complexidade. Quanto maior $\lambda$, maior o efeito de regularização na complexidade, isto é, mais penalizados os pesos muito grandes nas features são. Dessa forma, a nova função que se quer minimizar no modelo é dada por:

$f(\theta)$ = Loss + $\lambda$ Complexity = $\frac{1}{n} \sum_{i = 1}^{n} (\hat{y}_{i} - y_{i})^{2} + \lambda \sum_{i = 1}^{n} w_{i}^{2}$

cujas derivadas parciais são dadas por:

$\frac{\partial f}{\partial w_{k}} = \frac{2}{n}\sum_{i = 1}^{n} (\hat{y}_{i} - y_{i}) \cdot (x_{k})_{i} + 2\lambda w_{k}$, para toda feature $x_{k}$ com peso $w_{k}$.

$\frac{\partial f}{\partial b} = \frac{2}{n}\sum_{i = 1}^{n} (\hat{y}_{i} - y_{i})$

Assim, obtemos os novos valores do peso e do viés após uma iteração:

$w_{k}' = (1 - 2 \alpha \lambda)\;w_{k} - \frac{2 \alpha}{n}\sum_{i = 1}^{n} (\hat{y}_{i} - y_{i}) \cdot (x_{k})_{i}$, para toda feature $x_{k}$ com peso $w_{k}$.


$b' = b - \frac{2 \alpha}{n}\sum_{i = 1}^{n} (\hat{y}_{i} - y_{i})$

### Implementação

In [ ]:
class Regularization_Linear_Regression_Model():
    def __init__(self, train_data, features_name: list, label_name: str, ws: list, b: float, alpha: float, lambda_reg: float, random_state: int):
        self.train_data = train_data
        self.features = train_data[features_name]
        self.label = train_data[label_name]
        self.label_name = label_name # saving here because label as a series doesn't save it.
        for i in range(0, len(ws)):
            ws[i] = float(ws[i])
        self.ws = np.array(ws) # weights
        self.b = b
        self.alpha = alpha
        self.lambda_reg = lambda_reg
        self.rand = np.random.RandomState(random_state)
        
    def print_parameters(self):
        for i in range(1, len(self.ws) + 1):
            print(f'w{i} = {self.ws[i - 1]}')
        print (f'b = {self.b}')
        
    def get_parameters(self):
        i_vals = list(range(1, len(self.ws) + 1))
        parameters = {f'w{i}': self.ws[i - 1] for i in i_vals}
        parameters['b'] = self.b
        return parameters
        
    def get_single_prediction(self, xs: list):
        '''Get the prediction for a list with all the features' values.'''
        for i in range(0, len(xs)):
            xs[i] = float(xs[i])
        xs = np.array(xs)
        pred = np.dot(self.ws, xs) + self.b
        return pred
    
    def predict(self, data):
        n = len(data)
        features_name = self.features.columns
        data_features = data[features_name]
        predictions = np.zeros(n)
        for i in range(0, n):
            xs = list(data_features.iloc[i])
            predictions[i] += self.get_single_prediction(xs)
        return predictions
    
    def get_loss(self, data):
        n = len(data)
        data_label = data[self.label_name]
        y = np.array(data_label)
        predictions = self.predict(data)
        diff = predictions - y
        loss = (1/n)*np.dot(diff, diff)
        return loss
    
    def get_complexity(self):
        return np.dot(self.ws, self.ws)
    
    def get_structural_risk(self, data):
        return self.get_loss(data) + self.lambda_reg * self.get_complexity()
               
    def sgd_update_parameters(self, batch_size: int):
        n = len(self.label)
        index_list = list(range(0, n))
        random_indices = self.rand.choice(index_list, size = batch_size, replace = True) # bootstrap sample
        xs_sample = list()
        y_sample = np.array(self.label.iloc[random_indices])
        preds_sample = np.zeros(batch_size)
        for i in range(0, batch_size):
            xs = list(self.features.iloc[random_indices[i]])
            preds_sample[i] += self.get_single_prediction(xs)
        for col in self.features:
            xs_sample.append(np.array(self.features[col].iloc[random_indices])) # len(xs_sample) = len(self.ws)
        diff_sample = preds_sample - y_sample
        partial_w = np.zeros(len(self.ws))
        for i in range(0, len(self.ws)):
            partial_w[i] += (2/batch_size) * np.dot(diff_sample, xs_sample[i]) + 2*self.lambda_reg*self.ws[i]
        partial_b = (2/batch_size) * np.sum(diff_sample)
        self.ws -= self.alpha * partial_w
        self.b -= self.alpha * partial_b
        
    def sgd(self, iterations: int, batch_size: float, print_loss: bool): # stochastic gradient descent
        for i in range(0, iterations):
            self.sgd_update_parameters(batch_size)
            if print_loss:
                print(f'loss = {self.get_loss(self.train_data)}')
    
    @staticmethod
    def shuffle_data(data, random_state):
        rand = np.random.RandomState(random_state)
        return data.reindex(rand.permutation(data.index))
    
    @staticmethod
    def train_val_test_split(data, test_split: float, val_split: float):
        '''Get train, validation and test dataframes from data.'''
        n = len(data)
        test_size = int(test_split * n)
        val_size = int(val_split * n)
        test_data = data.iloc[list(range(0, test_size))]
        val_data = data.iloc[list(range(test_size, test_size + val_size))]
        train_data = data.iloc[list(range(test_size + val_size, n))]
        return train_data, val_data, test_data

## Treinando o modelo

In [ ]:
random_state = 0
data_shuffled = Regularization_Linear_Regression_Model.shuffle_data(data_scaled, random_state = random_state)

In [ ]:
train_data, val_data, test_data = Regularization_Linear_Regression_Model.train_val_test_split(data_shuffled, test_split = 0.2, val_split = 0.2)

### Vamos variar valores de lambda de 0 a 1 para verificar o que acontece com o custo e com a complexidade do modelo, mantendo os outros parâmetros constantes.

In [ ]:
lambda_reg_vals = np.arange(0, 1.1, 0.1)

In [ ]:
lambda_reg_vals

In [ ]:
ws = list(np.zeros(len(features)))
b = 0
alpha = 0.1
models = []
for lambda_reg in lambda_reg_vals:
    model = Regularization_Linear_Regression_Model(train_data = train_data, features_name = features,
                                        label_name = label, ws = ws, b = b, alpha = alpha, lambda_reg = lambda_reg,
                                        random_state = random_state)
    model.sgd(iterations = 100, batch_size = 10, print_loss = False)
    models.append(model)

In [ ]:
n = len(lambda_reg_vals)
i_vals = list(range(n))
lambda_row = {f'Model {i}': model.lambda_reg for (i, model) in zip(i_vals, models)}

In [ ]:
df_models = pd.DataFrame([lambda_row])
training_loss_row = {f'Model {i}': model.get_loss(train_data) for (i, model) in zip(i_vals, models)}
val_loss_row = {f'Model {i}': model.get_loss(val_data) for (i, model) in zip(i_vals, models)}
test_loss_row = {f'Model {i}': model.get_loss(test_data) for (i, model) in zip(i_vals, models)}
complexity_row = {f'Model {i}': model.get_complexity() for (i, model) in zip(i_vals, models)}

df_models = df_models.append(training_loss_row, ignore_index = True)
df_models = df_models.append(val_loss_row, ignore_index = True)
df_models = df_models.append(test_loss_row, ignore_index = True)
df_models = df_models.append(complexity_row, ignore_index = True)
df_models = df_models.rename(index ={0: 'Lambda', 1: 'Training Loss', 2: 'Validation Loss', 3: 'Test Loss', 4: 'Complexity'})

In [ ]:
df_models

Verificamos justamente o esperado:

- __Aumento do valor de Lambda $\rightarrow$ Diminuição da Complexidade e Aumento do Custo do Modelo em todos os dados.__

### Podemos verificar esse fato graficamente também:

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(lambda_row.values(), complexity_row.values())
plt.plot(lambda_row.values(), training_loss_row.values())
plt.plot(lambda_row.values(), validation_loss_row.values())
plt.plot(lambda_row.values(), test_loss_row.values())
plt.title('Complexity x Lambda')
plt.xlabel('Lambda')
plt.ylabel('Complexity')
plt.legend(['Complexity', 'Training Loss', 'Validation Loss', 'Test Loss'])

### Vamos adicionar também o cálculo do risco estrutural para cada modelo em cada conjunto de dados:

In [ ]:
training_sr_row = {f'Model {i}': model.get_structural_risk(train_data) for (i, model) in zip(i_vals, models)}
val_sr_row = {f'Model {i}': model.get_structural_risk(val_data) for (i, model) in zip(i_vals, models)}
test_sr_row = {f'Model {i}': model.get_structural_risk(test_data) for (i, model) in zip(i_vals, models)}
df_models = df_models.append(training_sr_row, ignore_index = True)
df_models = df_models.append(val_sr_row, ignore_index = True)
df_models = df_models.append(test_sr_row, ignore_index = True)

In [ ]:
df_models = df_models.rename(index ={0: 'Lambda', 1: 'Training Loss', 2: 'Validation Loss', 3: 'Test Loss', 4: 'Complexity',
                                    5: 'Training Structural Risk', 6: 'Validation Structural Risk', 7: 'Test Structural Risk'})

In [ ]:
df_models

- A análise dos riscos estruturais dos modelos nos indica a mesma resposta da análise dos valores de perda do modelo nos dados de treinamento, validação e de teste, que nos leva a crer que o modelo com $lambda = 0$ é o melhor, dado que possui menores perdas e tais valores nos três datasets são muito próximos, negando a possibilidade de overfitting nos dados de treino. Tal modelo também é aquele com os menores riscos estruturais.

### Verificando que de fato os pesos das features diminuem:

In [ ]:
i = 0
for model in models:
    print(f'Model {i}')
    model.print_parameters()
    print('\n')
    i += 1

### Verificando isso graficamente:

In [ ]:
ws_str = [f'w{i + 1}' for i in range(len(features))]

In [ ]:
ws_str

In [ ]:
ws = []
for w_str in ws_str:
    w_list = []
    for model in models:
        w_list.append(model.get_parameters()[w_str])
    ws.append(w_list)

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(0, len(features)):
    plt.plot(lambda_reg_vals, ws[i])
plt.title('Weights x Lambda')
plt.xlabel('Lambda')
plt.ylabel('Weight')

- Vemos que quando aumentamos suficientemente o valor de lambda, os pesos tendem a diminuir e a ficarem próximos de zero.

- Poderíamos analisar também a distribuição dos pesos em cada modelo, verificando que se aproxima de uma distribuição normal na medida em que aumentamos o valor de $\lambda$. No entanto, como há poucas features, a visualização de tal processo não é boa.